In [ ]:
!pip install selenium
!pip install BeautifulSoup4
!pip install requests
!pip install webdriver-manager
!pip install -q streamlit
!pip install spacy
!python -m spacy download pt_core_news_sm

In [ ]:
%%writefile app.py

import os
import streamlit as st
import pandas as pd

import numpy as np

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from PIL import Image

import re
import nltk

import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize



st.set_page_config(layout="centered", page_icon=":earth_americas:")

# Use a classe SessionState para personalização
class SessionState:
    def __init__(self):
        self.custom_primary_color = "#0E8388"
        self.custom_background_color = "#2C3333"
        self.custom_secondary_background_color = "#2E4F4F"
        self.custom_text_color = "#CBE4DE"
        self.custom_font = "monospace"

ss = SessionState()

# Defina o estilo da aplicação com as configurações personalizadas
custom_style = f"""
    <style>
    .stApp {{
        background-color: {ss.custom_background_color};
        color: {ss.custom_text_color};
        font-family: {ss.custom_font};
    }}
    .stButton {{
        background-color: {ss.custom_primary_color};
    }}
    .css-3ewlls {{
        background-color: {ss.custom_secondary_background_color};
    }}
    </style>
"""

st.markdown(custom_style, unsafe_allow_html=True)

# Define o título da página com uma classe CSS personalizada
st.write("""
    <style>
    .pageTitle {
        color: #f0f0f0;
        text-align: center;
    }
    </style>
    <h1 class="pageTitle">GastronoTripReview</h1>
""", unsafe_allow_html=True)


st.header("",divider='green')


st.markdown(
    """
    <p style="color: white; font-size: 20px;">Consulte como está a reputação do seu restaurante através de um resumo fácil e prático decorrente da análise de sentimentos de reviews feitas por <strong style="font-weight: bold;">turistas e clientes reais</strong> na plataforma TripAdvisor. <span class="emoji">&#x1F5FA;</span> <span class="emoji">&#x1F374;</span></p>
    """,
    unsafe_allow_html=True,
)

image_url1 = "https://raw.githubusercontent.com/isalbineli/testes/main/smileys.jpg"

st.image(image_url1, use_column_width=True)

st.markdown(
    """
    <style>
    .sidebar .sidebar-content {
        background-color: #5c4f5c;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

image_url2 = "https://raw.githubusercontent.com/isalbineli/testes/main/shutterstock_1756325360.jpg"


st.sidebar.image(image_url2, caption="Suas reviews como você nunca viu :)")

# Adiciona outros elementos na barra lateral
#st.sidebar.title("Barra Lateral Personalizada")
#st.sidebar.text("Este é um exemplo de barra lateral personalizada.")


#:world_map: :knife_fork_plate:



def crawler(input_pesquisa):

  servico = Service(ChromeDriverManager().install())
  navegador = webdriver.Chrome(service=servico)

  navegador.get("https://www.tripadvisor.com/Restaurants")



  #input_pesquisa = "Pizzaria Camelo"

  # Encontre o campo de pesquisa
  search_box = navegador.find_element(By.XPATH,"/html/body/div[2]/div/div[1]/div/div/div/div/div/div[2]/div/div/div[2]/div/form/div/div/input")

  # Digite a consulta que você deseja pesquisar
  search_box.send_keys(input_pesquisa)

  # Envie a consulta pressionando Enter
  search_box.send_keys(Keys.RETURN)

  # Espere um tempo para que os resultados carreguem
  navegador.implicitly_wait(10)

  elemento = navegador.find_element(By.CLASS_NAME, "result-title")
  print(elemento)

  navegador.execute_script("arguments[0].click();", elemento)

  navegador.implicitly_wait(10)

  janelas = navegador.window_handles

  navegador.switch_to.window(janelas[0])

  navegador.implicitly_wait(10)

  navegador.close()

  navegador.switch_to.window(janelas[1])

  navegador.implicitly_wait(10)


  label_element = navegador.find_element(By.CSS_SELECTOR,'label[for="filters_detail_language_filterLang_pt"]')

  # Clique no elemento label
  label_element.click()


  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}


  link = navegador.current_url

  requisicao = requests.get(link, headers = headers)
  print(requisicao)

  #print(requisicao.text)
  site = BeautifulSoup(requisicao.text,"html.parser")

  print(site.prettify())

  if requisicao.status_code == 200:
      # Parse o conteúdo HTML da página
      soup = BeautifulSoup(requisicao.text, 'html.parser')

      # Encontre os elementos que contêm as avaliações (os seletores CSS podem variar)
      reviews = soup.find_all('div', class_='entry')

      avaliacoes = []

      # Itere pelas avaliações e extraia as informações desejadas
      for review in reviews:
          # Extraia o texto da avaliação
          review_text = review.find('p', class_='partial_entry').text
          avaliacoes.append(review_text)

          #print("Avaliação:", review_text)

      print(avaliacoes)
      return(avaliacoes)

  else:
      print("Falha ao acessar a página:", requisicao.status_code)


  navegador.quit()


def analisa_sentimento(texto):

  caminho_modelo = '/content/drive/MyDrive/MODELOS/best_model_state_ambiente.bin'

  modelo = BertForSequenceClassification.from_pretrained(caminho_modelo)

  tokenizer = BertTokenizer.from_pretrained(caminho_modelo)

  analisar(texto)



def prepara_dados_analise(pesquisa):

  nome_coluna = ['comentario']

  dataframe = pd.DataFrame(pesquisa, columns=nome_coluna)

  # Converter todas a letras para minúsculas
  dataframe['comentario'] = dataframe['comentario'].str.lower()

  # Remover espaços extras
  dataframe['comentario'] = dataframe['comentario'].apply(lambda x: x.strip())

  texto = dataframe['comentario'].tolist()

  return texto


#movendo tensores para algum dispositivo GPU ou CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def classify_sentiment(text):
    encoded_text = tokenizer.encode_plus(
        text,
        max_length=max_len,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='longest',
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoded_text['input_ids'].to(device)
    attention_mask = encoded_text['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    logits = output.logits
    probabilities = torch.softmax(logits, dim=1)
    sentiment_probs = probabilities[0].tolist()
    sentiment_index = torch.argmax(logits, dim=1).item()
    sentiment_label = class_names[sentiment_index]

    return sentiment_probs, sentiment_label

def analisar(texto):

  colunas = ['Comentário', 'Sentimento']
  dataframe_resultados = pd.DataFrame(columns=colunas)


  for i in texto:

      user_text = i

      # analisa os sentimentos
      sentiment_probs, sentiment_label = classify_sentiment(user_text)
      sentiment_labels = pd.DataFrame(sentiment_probs, class_names)[0]

      tabela_para_concatenar = pd.Series(sentiment_labels).to_frame().transpose()

      # Adicionar uma linha ao DataFrame
      nova_linha = {'Comentário': {user_text}, 'Sentimento': {sentiment_label}}
      dataframe_resultados = dataframe_resultados.append(nova_linha, ignore_index=True)
      dataframe_resultados =  pd.concat([dataframe_resultados, tabela_para_concatenar], axis=1)

      return dataframe_resultados




with st.form("chat_form"):
    st.markdown(
    """
    <p style="color: white;">Insira o nome do restaurante para consulta:</p>
    """,
    unsafe_allow_html=True,
)
    input_pesquisa = st.text_input("")
    submit_button = st.form_submit_button("Analisar",type="primary",use_container_width=True)



if submit_button:
  pesquisa = crawler(input_pesquisa)

  texto = prepara_dados_analise(pesquisa)

  df_res_sentimento = analisa_sentimento(texto)



Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.230.4.61


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 4.41s
your url is: https://hip-apples-sin.loca.lt
^C
